In [1]:
# check the powershell host environment
(Get-Host).Version


Major  Minor  Build  Revision
-----  -----  -----  --------
7      2      8      -1



In [2]:
<#
    Function that starts an Az Data Factory Pipeline and waits until the pipeline finishes execution.
#>
function Run-AzDataFactoryV2PipelineSync {
    param (
        [Parameter(Mandatory)]
        [string] $resourceGroupName,
        [string] $dataFactoryName,
        [string] $pipelineName
    ) 

    $runID = Invoke-AzDataFactoryV2Pipeline `
        -ResourceGroupName $resourceGroupName `
        -DataFactoryName   $dataFactoryName `
        -PipelineName      $pipelineName

    Write-Host "Started pipeline $pipelineName with RunID: $runID"

    $runInfo = $null 

    do {
        Start-Sleep -Seconds 15

        $runInfo = Get-AzDataFactoryV2PipelineRun `
            -ResourceGroupName $resourceGroupName `
            -DataFactoryName   $dataFactoryName `
            -PipelineRunId     $runID

    } while ($runInfo.RunEnd -eq $null)

    Write-Host "Pipeline finished with status $($runInfo.Status)"
}

In [3]:
# install the PS SQLServer module
# Install-Module -Name SqlServer

# install the PS Azure module
# Install-Module -Name Az -Scope CurrentUser -Repository PSGallery -Force

# check that sqlcmd is installed
# sqlcmd -?

In [3]:
# set the parameters

$randomString = "rgawltdw" + $(get-random)

$location = "EastUS"
$resourceGroupName = "$randomString" + "rg"

$serverName = "$resourceGroupName-sqlserver"
$databaseName = "$resourceGroupName-sqldb"
$server = "$serverName.database.windows.net"
$sqlLogin = "dbadmin"
# generate a random password for the SQL Server Login
$password = "VjK" + $(get-random) + "@PS"

$adlsAccountName = "$resourceGroupName" + "dls"
$adlsURL = "https://$adlsAccountName.dfs.core.windows.net/"

$dataFactoryName = "$resourceGroupName-df"

$keyVaultName = "$resourceGroupName-kv"
$keyVaultURL = "https://$keyVaultName.vault.azure.net/"

# add your IP address to the sql server allowed IPs list
$myIP = "0.0.0.0"

. {
>> # set the parameters
>> 
>> $randomString = "rgawltdw" + $(get-random)
>> 
>> $location = "EastUS"
>> $resourceGroupName = "$randomString" + "rg"
>> 
>> $serverName = "$resourceGroupName-sqlserver"
>> $databaseName = "$resourceGroupName-sqldb"
>> $server = "$serverName.database.windows.net"
>> $sqlLogin = "dbadmin"
>> # generate a random password for the SQL Server Login
>> $password = "VjK" + $(get-random) + "@PS"
>> 
>> $adlsAccountName = "$resourceGroupName" + "dls"
>> $adlsURL = "https://$adlsAccountName.dfs.core.windows.net/"
>> 
>> $dataFactoryName = "$resourceGroupName-df"
>> 
>> $keyVaultName = "$resourceGroupName-kv"
>> $keyVaultURL = "https://$keyVaultName.vault.azure.net/"
>> 
>> # add your IP address to the sql server allowed IPs list
>> $myIP = "0.0.0.0"
>> }
>> 


In [4]:
# connect to Azure
$null = Connect-AzAccount

## Deploy Resources

In [5]:
# create a resource group
$null = New-AzResourceGroup `
    -Name $resourceGroupName `
    -Location $location 

In [6]:
# create an adls storage account
$null = New-AzStorageAccount `
    -ResourceGroupName $resourceGroupName `
    -Name $adlsAccountName `
    -Type Standard_LRS `
    -Location $location `
    -AccessTier Cool `
    -EnableHierarchicalNamespace $true

In [7]:
# create an sql server
$passwordSecureString = ConvertTo-SecureString `
    -String $password -AsPlainText -Force

$sqlCredentials = New-Object `
    -TypeName System.Management.Automation.PSCredential `
    -ArgumentList $sqlLogin, $passwordSecureString

$null = New-AzSqlServer `
    -ResourceGroupName $resourceGroupName `
    -ServerName $serverName `
    -Location $location `
    -SqlAdministratorCredentials $sqlCredentials

# allow azure services to connect to the sql server
$null = New-AzSqlServerFirewallRule `
    -ResourceGroupName $resourceGroupName `
    -ServerName $serverName `
    -AllowAllAzureIPs

# create a server firewall rule that allows access to your ip
$null = New-AzSqlServerFirewallRule `
    -ResourceGroupName $resourceGroupName `
    -ServerName $serverName `
    -FirewallRuleName "AllowedIPs" `
    -StartIpAddress $myIP `
    -EndIpAddress $myIP

. {
>> # create an sql server
>> $passwordSecureString = ConvertTo-SecureString `
>>     -String $password -AsPlainText -Force
>> 
>> $sqlCredentials = New-Object `
>>     -TypeName System.Management.Automation.PSCredential `
>>     -ArgumentList $sqlLogin, $passwordSecureString
>> 
>> $null = New-AzSqlServer `
>>     -ResourceGroupName $resourceGroupName `
>>     -ServerName $serverName `
>>     -Location $location `
>>     -SqlAdministratorCredentials $sqlCredentials
>> 
>> # allow azure services to connect to the sql server
>> $null = New-AzSqlServerFirewallRule `
>>     -ResourceGroupName $resourceGroupName `
>>     -ServerName $serverName `
>>     -AllowAllAzureIPs
>> 
>> # create a server firewall rule that allows access to your ip
>> $null = New-AzSqlServerFirewallRule `
>>     -ResourceGroupName $resourceGroupName `
>>     -ServerName $serverName `
>>     -FirewallRuleName "AllowedIPs" `
>>     -StartIpAddress $myIP `
>>     -EndIpAddress $myIP
>> }
>> 


In [8]:
# create a serverless sql database with the AdventureWorksLT sample
$null = New-AzSqlDatabase  `
    -ResourceGroupName       $resourceGroupName `
    -ServerName              $serverName `
    -DatabaseName            $databaseName  `
    -ComputeModel            Serverless `
    -Edition                 GeneralPurpose `
    -Vcore                   1 `
    -AutoPauseDelayInMinutes 60 `
    -BackupStorageRedundancy Local `
    -MaxSizeBytes            2GB `
    -ComputeGeneration       Gen5 `
    -SampleName              AdventureWorksLT

- The output type 'Microsoft.Azure.Commands.Sql.Database.Model.AzureSqlDatabaseModel' is changing
- The following properties in the output type are being deprecated : 'BackupStorageRedundancy'
- The following properties are being added to the output type : 'CurrentBackupStorageRedundancy' 'RequestedBackupStorageRedundancy'
- The change is expected to take effect from the version : '3.0.0'
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.


In [9]:
# create a key vault
$null = New-AzKeyVault `
    -Name $keyVaultName `
    -ResourceGroupName $resourceGroupName `
    -Location $location

# create a connection string to the AdventureWorksLT database
$connectionStringSS = (
    "Data Source=$server;" + 
    "Initial Catalog=$databaseName;" + 
    "Integrated Security=False;" + 
    "User ID=$sqlLogin;" +
    "Password=$password"
    ) | ConvertTo-SecureString -AsPlainText -Force

# create a secret for the AdventureWorksLT connection string
$null = Set-AzKeyVaultSecret `
    -VaultName $keyVaultName `
    -Name "advworkslt-sqldb-cs" `
    -SecretValue $connectionStringSS

# get the storage key 
$storageKeySecureString = (
    Get-AzStorageAccountKey `
    -ResourceGroupName $resourceGroupName `
    -Name $adlsAccountName |
    Where-Object {$_.KeyName -eq "key1"}).Value | 
    ConvertTo-SecureString -AsPlainText -Force

# create a secret for adls storage key
$null = Set-AzKeyVaultSecret `
    -VaultName $keyVaultName `
    -Name "advworkslt-adls-ak" `
    -SecretValue $storageKeySecureString

Visit https://go.microsoft.com/fwlink/?linkid=2181475 for any permission issues.


In [10]:
# create a data factory
$dataFactory = New-AzDataFactoryV2 `
    -ResourceGroupName   $resourceGroupName `
    -Location            $location `
    -Name                $dataFactoryName

# parametrize the adf resources
Get-Content .\adf-resources\deployment\adf_config_template.csv | 
    ForEach-Object { $ExecutionContext.InvokeCommand.ExpandString($_) } > .\adf-resources\deployment\adf_config.csv
    
# set the options of the deployment
$opt = New-AdfPublishOption
$opt.DeleteNotInSource = $true

# deploy the adf-resources to the data factory
$null = Publish-AdfV2FromJson `
    -RootFolder            .\adf-resources `
    -ResourceGroupName     $resourceGroupName `
    -DataFactoryName       $dataFactoryName `
    -Location              $location `
    -Option                $opt `
    -Stage                 .\adf-resources\deployment\adf_config.csv

# grant secret list and get permissions to the data factory
$null = Set-AzKeyVaultAccessPolicy `
    -VaultName $keyVaultName `
    -ObjectId $dataFactory.Identity.PrincipalId `
    -PermissionsToSecrets get,list -PassThru

### azure.datafactory.tools                                       Version 0.110.001 ###
Invoking Publish-AdfV2FromJson (https://github.com/SQLPlayer/azure.datafactory.tools)
with the following parameters:
RootFolder:         .\adf-resources
ResourceGroupName:  rgawltdw395887366rg
DataFactoryName:    rgawltdw395887366rg-df
Location:           EastUS
Stage:              .\adf-resources\deployment\adf_config.csv
Options provided:   True
Publishing method:  AzResource
Is Dry Run?:        False
Publish options are provided.
STEP: Verifying whether ADF exists...
Azure Data Factory exists.
STEP: Reading Azure Data Factory from JSON files...
IntegrationRuntimes: 1 object(s) loaded.
LinkedServices: 3 object(s) loaded.
Pipelines: 5 object(s) loaded.
DataSets: 12 object(s) loaded.
DataFlows: 4 object(s) loaded.
Triggers: 0 object(s) loaded.
Managed VNet: 0 object(s) loaded.
Managed Private Endpoints: 0 object(s) loaded.
Credentials: 0 object(s) loaded.
Factories: 2 object(s) loaded.
# Number of o

## Simulate operations

In [13]:
# run the db setup
sqlcmd -S $server -d $databaseName  -U $sqlLogin -P $password -I -i db-scripts/1_database_setup.sql

# activate change tracking
sqlcmd -S $server -d $databaseName  -U $sqlLogin -P $password -I -i db-scripts/2_activate_change_tracking.sql -v DATABASE=$databaseName

# create the integration schema
sqlcmd -S $server -d $databaseName  -U $sqlLogin -P $password -I -i db-scripts/3_create_integration_schema.sql

# create the presentation schema
sqlcmd -S $server -d $databaseName  -U $sqlLogin -P $password -I -i db-scripts/3_create_presentation_schema.sql

SUCCESSFULLY MODIFIED DATABASE
CHANGE TRACKING ACTIVATED SUCCESSFULLY
INTEGRATION SCHEMA CREATED SUCCESSFULLY
PRESENTATION SCHEMA CREATED SUCCESSFULLY


In [16]:
# run the customer history initial load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName CustomerHistoryInitialLoad

Started pipeline CustomerHistoryInitialLoad with RunID: 38ed19a6-c60f-4ebb-8842-fbaefd013575
Pipeline finished with status Succeeded


In [17]:
# run the product history initial load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName ProductHistoryInitialLoad 

Started pipeline ProductHistoryInitialLoad with RunID: 34a695dc-4739-4c20-875e-e018caa06a2f
Pipeline finished with status Succeeded


In [18]:
# run the sales orders initial load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName SalesOrdersIncrLoad 

Started pipeline SalesOrdersIncrLoad with RunID: 012c2cbb-d226-4470-af4d-181bda60ed7a
Pipeline finished with status Succeeded


In [14]:
# check the job logs table

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT * 
FROM integration.JobLogs
ORDER BY pipeline_name, sync_timestamp
"


pipeline_name pipeline_run_id                      sync_ct_version sync_timestamp
------------- ---------------                      --------------- --------------
customer_sync 369a9934-ee1e-4564-b4ba-96bd1511cbbe               0 8/01/2023 5:26:49 p. m.



In [15]:
# check the number of rows for tables in the presentation schema

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
    (SELECT COUNT(*) FROM presentation.CustomersHistory) as CustomersHistoryNRows,
    (SELECT COUNT(*) FROM presentation.ProductsHistory) as ProductsHistoryNRows,
    (SELECT COUNT(*) FROM presentation.FactSalesOrders) as FactSalesOrdersNRows
"


CustomersHistoryNRows ProductsHistoryNRows FactSalesOrdersNRows
--------------------- -------------------- --------------------
                   32                    0                    0



In [16]:
# execute the operations of day 1
sqlcmd -S $server -d $databaseName  -U $sqlLogin -P $password -I -i db-scripts/4_operations_day_1.sql

SUCCESSFULLY COMPLETED OPERATIONS FOR DAY 1


In [22]:
# run the customer history incremental load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName CustomerHistoryIncrLoad

Started pipeline CustomerHistoryIncrLoad with RunID: 34acf61e-d3b0-426f-bee2-4e5a2250cc88
Pipeline finished with status Succeeded


In [23]:
# run the product history incremental load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName ProductHistoryIncrLoad

Started pipeline ProductHistoryIncrLoad with RunID: 45b75887-1064-4521-9e39-4f0ba323ab34
Pipeline finished with status Succeeded


In [24]:
# run the sales orders incremental load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName SalesOrdersIncrLoad

Started pipeline SalesOrdersIncrLoad with RunID: abefc3c9-518d-4166-a0d1-c8cc07fdaee8
Pipeline finished with status Succeeded


In [24]:
# check the job logs table

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT * 
FROM integration.JobLogs
ORDER BY pipeline_name, sync_timestamp
"


pipeline_name pipeline_run_id                      sync_ct_version sync_timestamp
------------- ---------------                      --------------- --------------
customer_sync 369a9934-ee1e-4564-b4ba-96bd1511cbbe               0 8/01/2023 5:26:49 p. m.
customer_sync d1ec0254-9e90-4c0e-b1bc-619c56d37816               7 8/01/2023 5:48:41 p. m.
customer_sync 869e1f40-e474-4545-a756-1f66b9843972               7 8/01/2023 5:50:22 p. m.



In [25]:
# check the number of rows for tables in the presentation schema
# the number of rows in the CustomersHistory and ProductsHistory table should increment
# the number of rows in the FactSalesOrders should not increment as no orders have 
#     reached finished state since the last syncrhonization

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
    (SELECT COUNT(*) FROM presentation.CustomersHistory) as CustomersHistoryNRows,
    (SELECT COUNT(*) FROM presentation.ProductsHistory) as ProductsHistoryNRows,
    (SELECT COUNT(*) FROM presentation.FactSalesOrders) as FactSalesOrdersNRows
"


CustomersHistoryNRows ProductsHistoryNRows FactSalesOrdersNRows
--------------------- -------------------- --------------------
                   35                    0                    0



In [19]:
# check the changes made by transaction number 1
# check that the new customer (Eugene Porter) has been loaded to the DW

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT * 
FROM presentation.DimCustomer 
WHERE FirstName = 'Eugene' and LastName = 'Porter'
ORDER BY RowEffectiveDate;
"


SurrogateKey            : 35
CustomerID              : 30119
NameStyle               : False
Title                   : 
FirstName               : Eugene
MiddleName              : 
LastName                : Porter
Suffix                  : 
CompanyName             : Regressive Sports
EmailAddress            : eugene0@adventure-works.com
Phone                   : 279-555-0130
SalesPerson             : adventure-works\jillian0
MainOfficeAddressLine1  : 1434 Marshall Rd
MainOfficeAddressLine2  : 
MainOfficeCity          : Alpine
MainOfficeStateProvince : California
MainOfficeCountryRegion : United States
MainOfficePostalCode    : 91901
RowEffectiveDate        : 8/01/2023 5:27:57 p. m.
RowExpirationDate       : 31/12/9999 12:00:00 a. m.
RowCurrentFlag          : True
RowDeletedFlag          : False




In [22]:
# check the changes made by transaction number 2
# check that the sales person of customer 29568 has been modified
# given that this is an SCD 2 column, a new row should be inserted.

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
CustomerID, SalesPerson, 
RowEffectiveDate, RowExpirationDate, RowCurrentFlag as RCF, RowDeletedFlag as RDF
FROM presentation.DimCustomer 
WHERE CustomerID = 29568
ORDER BY RowEffectiveDate;
" | 
Format-Table


CustomerID SalesPerson            RowEffectiveDate         RowExpirationDate           RCF   RDF
---------- -----------            ----------------         -----------------           ---   ---
     29568 adventure-works\shu0   1/06/2008 12:00:00 a. m. 8/01/2023 5:27:57 p. m.   False False
     29568 adventure-works\linda3 8/01/2023 5:27:57 p. m.  31/12/9999 12:00:00 a. m.  True False



In [23]:
# check the changes made by transaction number 3
# check that the main office address line 1 of customer 29485 has been modified
# given that these are an SCD 2 column, a new row should be inserted

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
CustomerID, MainOfficeAddressLine1, 
RowEffectiveDate, RowExpirationDate, RowCurrentFlag as RCF, RowDeletedFlag as RDF
FROM presentation.DimCustomer
WHERE CustomerID = 29485
ORDER BY RowEffectiveDate;
" | 
Format-Table


CustomerID MainOfficeAddressLine1 RowEffectiveDate         RowExpirationDate           RCF   RDF
---------- ---------------------- ----------------         -----------------           ---   ---
     29485 57251 Serene Blvd      1/06/2008 12:00:00 a. m. 8/01/2023 5:27:57 p. m.   False False
     29485 7343 Sepulveda Blvd    8/01/2023 5:27:57 p. m.  31/12/9999 12:00:00 a. m.  True False



In [21]:
# check the changes made by transaction number 6
# check the list price of products with product model 36 (Touring-3000) have been modified
# given that this is an SCD 2 column, a new row should be inserted for each product

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
ProductID, ProductModel, ListPrice,
RowEffectiveDate, RowExpirationDate, RowCurrentFlag as RCF, RowDeletedFlag as RDF
FROM presentation.DimProduct
WHERE ProductModel = 'Touring-3000'
ORDER BY ProductID, RowEffectiveDate;
" | 
Format-Table

In [31]:
# check the changes made by transaction number 7
# check that the product category 'Bottles and Cages' has been divided into 'Bottles' and 'Bottle Cages'
# given that this is an SCD 1 column, the existing rows should be modified

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
ProductID, ProductSubcategory, 
RowEffectiveDate, RowExpirationDate, RowCurrentFlag as RCF, RowDeletedFlag as RDF
FROM presentation.DimProduct
WHERE ProductID IN (870, 871, 872)
ORDER BY ProductID, RowEffectiveDate;
" | 
Format-Table


ProductID ProductSubcategory RowEffectiveDate         RowExpirationDate           RCF   RDF
--------- ------------------ ----------------         -----------------           ---   ---
      870 Bottles            1/07/2007 12:00:00 a. m. 2/01/2023 12:47:44 p. m.  False False
      870 Bottles            2/01/2023 12:47:44 p. m. 31/12/9999 12:00:00 a. m.  True False
      871 Bottle Cages       1/07/2007 12:00:00 a. m. 2/01/2023 12:47:44 p. m.  False False
      871 Bottle Cages       2/01/2023 12:47:44 p. m. 31/12/9999 12:00:00 a. m.  True False
      872 Bottle Cages       1/07/2007 12:00:00 a. m. 2/01/2023 12:47:44 p. m.  False False
      872 Bottle Cages       2/01/2023 12:47:44 p. m. 31/12/9999 12:00:00 a. m.  True False



In [32]:
# check the changes made by transaction number 8
# check that product 907 has been discontinued
# given that this is an SCD 1 column, the SellEndDate should be updated for all rows

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
ProductID, SellEndDate, 
RowEffectiveDate, RowExpirationDate, RowCurrentFlag as RCF, RowDeletedFlag as RDF
FROM presentation.DimProduct 
WHERE ProductID = 907
ORDER BY ProductID, RowEffectiveDate;
" | 
Format-Table


ProductID SellEndDate              RowEffectiveDate         RowExpirationDate           RCF   RDF
--------- -----------              ----------------         -----------------           ---   ---
      907 2/01/2023 12:47:44 p. m. 1/07/2007 12:00:00 a. m. 2/01/2023 12:47:44 p. m.  False False
      907 2/01/2023 12:47:44 p. m. 2/01/2023 12:47:44 p. m. 31/12/9999 12:00:00 a. m.  True False



In [33]:
# check the changes made by transaction number 9
# check that the name of product model 1 has changed from 'Classic Vest' to 'Standard Vest'
# given that this is an SCD 1 column, all rows should be modified

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
ProductID, ProductModel,
RowEffectiveDate, RowExpirationDate, RowCurrentFlag as RCF, RowDeletedFlag as RDF
FROM presentation.DimProduct
WHERE ProductID IN (864, 865, 866)
ORDER BY ProductID, RowEffectiveDate;
" | 
Format-Table


ProductID ProductModel  RowEffectiveDate         RowExpirationDate           RCF   RDF
--------- ------------  ----------------         -----------------           ---   ---
      864 Standard Vest 1/07/2007 12:00:00 a. m. 2/01/2023 12:47:44 p. m.  False False
      864 Standard Vest 2/01/2023 12:47:44 p. m. 31/12/9999 12:00:00 a. m.  True False
      865 Standard Vest 1/07/2007 12:00:00 a. m. 2/01/2023 12:47:44 p. m.  False False
      865 Standard Vest 2/01/2023 12:47:44 p. m. 31/12/9999 12:00:00 a. m.  True False
      866 Standard Vest 1/07/2007 12:00:00 a. m. 2/01/2023 12:47:44 p. m.  False False
      866 Standard Vest 2/01/2023 12:47:44 p. m. 31/12/9999 12:00:00 a. m.  True False



In [34]:
# execute the operations of day 2
sqlcmd -S $server -d $databaseName  -U $sqlLogin -P $password -I -i db-scripts/4_operations_day_2.sql

SUCCESSFULLY COMPLETED OPERATIONS FOR DAY 2


In [35]:
# run the customer history incremental load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName CustomerHistoryIncrLoad

Started pipeline CustomerHistoryIncrLoad with RunID: cdb9608e-bda8-4606-80ce-89c5f73aae7c
Pipeline finished with status Succeeded


In [36]:
# run the product history incremental load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName ProductHistoryIncrLoad

Started pipeline ProductHistoryIncrLoad with RunID: 703fc21a-7e1e-4157-b4d6-c46d6050275e
Pipeline finished with status Succeeded


In [37]:
# run the sales orders incremental load
Run-AzDataFactoryV2PipelineSync $resourceGroupName $dataFactoryName SalesOrdersIncrLoad

Started pipeline SalesOrdersIncrLoad with RunID: 93d552e2-c2ef-41a4-8776-745321ffd83c
Pipeline finished with status Succeeded


In [38]:
# check the job logs table

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT * 
FROM integration.JobLogs
ORDER BY pipeline_name, sync_timestamp
"


pipeline_name    pipeline_run_id                      sync_ct_version sync_timestamp
-------------    ---------------                      --------------- --------------
customer_sync    38ed19a6-c60f-4ebb-8842-fbaefd013575               0 2/01/2023 12:39:16 p. m.
customer_sync    34acf61e-d3b0-426f-bee2-4e5a2250cc88               7 2/01/2023 12:48:04 p. m.
customer_sync    cdb9608e-bda8-4606-80ce-89c5f73aae7c               7 2/01/2023 12:53:50 p. m.
product_sync     34a695dc-4739-4c20-875e-e018caa06a2f               0 2/01/2023 12:44:12 p. m.
product_sync     45b75887-1064-4521-9e39-4f0ba323ab34               7 2/01/2023 12:49:40 p. m.
product_sync     703fc21a-7e1e-4157-b4d6-c46d6050275e               7 2/01/2023 12:55:17 p. m.
sales_order_sync 012c2cbb-d226-4470-af4d-181bda60ed7a                 2/01/2023 12:45:06 p. m.
sales_order_sync abefc3c9-518d-4166-a0d1-c8cc07fdaee8                 2/01/2023 12:50:48 p. m.
sales_order_sync 93d552e2-c2ef-41a4-8776-745321ffd83c                

In [39]:
# check the number of rows for tables in the presentation schema
# the number of rows in the CustomersHistory and ProductsHistory tables should not increment
# the number of rows in FactSalesOrdersRows should increment by 3

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
    (SELECT COUNT(*) FROM presentation.CustomersHistory) as CustomersHistoryNRows,
    (SELECT COUNT(*) FROM presentation.ProductsHistory) as ProductsHistoryNRows,
    (SELECT COUNT(*) FROM presentation.FactSalesOrders) as FactSalesOrdersNRows
"


CustomersHistoryNRows ProductsHistoryNRows FactSalesOrdersNRows
--------------------- -------------------- --------------------
                   35                  312                  545



In [40]:
# check the changes made by transaction number 1
# check that the sales order placed by customer 29781 has been loaded into the DW

Invoke-Sqlcmd -ServerInstance $server -Database $databaseName -Username $sqlLogin -Password $password -Query "
SELECT 
SalesOrderID, SalesOrderDetailID, OrderDate, CustomerID, ProductID
FROM presentation.FactSalesOrders
WHERE CustomerID = 29781
" | 
Format-Table


SalesOrderID SalesOrderDetailID OrderDate                CustomerID ProductID
------------ ------------------ ---------                ---------- ---------
           1             113407 2/01/2023 12:47:44 p. m.      29781       870
           1             113408 2/01/2023 12:47:44 p. m.      29781       874
           1             113409 2/01/2023 12:47:44 p. m.      29781       875
       71923             113152 1/06/2008 12:00:00 a. m.      29781       870
       71923             113153 1/06/2008 12:00:00 a. m.      29781       874
       71923             113154 1/06/2008 12:00:00 a. m.      29781       875



In [41]:
# delete the resource group
Remove-AzResourceGroup `
    -name  $resourceGroupName `
    -Force

True
